# Run in Colab

## Test 11

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
##

## Test2

In [3]:
%pip install deepface
%pip install ethnicolr

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 43.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 kB 13.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.2/80.2 kB 11.0 MB/s eta 0:00:00
  Created wheel for fire: filename=fire-0.5.0-py2.py3-none-any.whl size=116932 sha256=bf76f6cf07b6a038379b67fc486d27cc5ca727b3808eec3fe7fc4ebb5cb9ebbf
  Stored in directory: /root/.cache/pip/wheels/90/d4/f7/9404e5db0116bd4d43e5666eaa3e70ab53723e1e3ea40c9a95
Successfully built fire
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.5/39.5 MB 19.9 MB/s eta 0:00:00


## Test 3

In [4]:
from deepface import DeepFace

# specify the image path
img_path = 'https://image.tmdb.org/t/p/w300_and_h450_bestv2/ptCynXMskQ8CETCvg6ZjSwcMyVI.jpg'
img_path = 'https://www.themoviedb.org/t/p/w300_and_h450_bestv2/jj2Gcobpopokal0YstuCQW0ldJ4.jpg'
# use DeepFace to analyze the image, setting enforce_detection to False
results = DeepFace.analyze(img_path, actions = ['age', 'gender', 'race', 'emotion'], enforce_detection = False)

# assuming the first item in the results list pertains to your image
result = results[0]

# print the results
print("Age: ", result["age"])
print("Gender: ", result["gender"])
print("Race: ", result["dominant_race"])
print("Emotion: ", result["dominant_emotion"])


Directory  /root /.deepface created
Directory  /root /.deepface/weights created
facial_expression_model_weights.h5 will be downloaded...


Downloading...
From: https://github.com/serengil/deepface_models/releases/download/v1.0/facial_expression_model_weights.h5
To: /root/.deepface/weights/facial_expression_model_weights.h5
100%|██████████| 5.98M/5.98M [00:00<00:00, 80.6MB/s]


age_model_weights.h5 will be downloaded...


Downloading...
From: https://github.com/serengil/deepface_models/releases/download/v1.0/age_model_weights.h5
To: /root/.deepface/weights/age_model_weights.h5
100%|██████████| 539M/539M [00:00<00:00, 553MB/s]


gender_model_weights.h5 will be downloaded...


Downloading...
From: https://github.com/serengil/deepface_models/releases/download/v1.0/gender_model_weights.h5
To: /root/.deepface/weights/gender_model_weights.h5
100%|██████████| 537M/537M [00:29<00:00, 18.1MB/s]


race_model_single_batch.h5 will be downloaded...


Downloading...
From: https://github.com/serengil/deepface_models/releases/download/v1.0/race_model_single_batch.h5
To: /root/.deepface/weights/race_model_single_batch.h5
100%|██████████| 537M/537M [00:00<00:00, 554MB/s]
Action: emotion: 100%|██████████| 4/4 [00:08<00:00,  2.08s/it]

Age:  29
Gender:  {'Woman': 0.013050185225438327, 'Man': 99.98694658279419}
Race:  black
Emotion:  neutral


In [5]:
### Google Drive opens
### Continue where left off from previous text if runtime stops
### To save time, write file every 100 rows

import os
import pandas as pd
from deepface import DeepFace
from ethnicolr import pred_census_ln
import requests
from PIL import Image
from io import BytesIO
from google.colab import drive

# Mount your Google Drive
drive.mount('/content/drive')

# Path to the CSV file on Google Drive
file_path = '/content/drive/MyDrive/Projects/scrotten_movies/'
read_file = 'person_data.csv'

# Read the CSV file into a DataFrame
data = pd.read_csv(file_path+read_file)

# counter variables
total_persons = len(data)
persons_analyzed = 0

def process_row(row, results_df):
    # Check if the race for this person has been previously determined
    if not results_df[results_df['id'] == row['id']].empty:
        return results_df

    result_dict = {
        'id': row['id'],
        'name': row['name'],
        'profile_path': row['profile_path'],
        'profile_race': None,
        'name_race': None,
    }

    # Infer race from image
    try:
        if pd.notna(row['profile_path']):
            img_path = f'https://image.tmdb.org/t/p/w300_and_h450_bestv2/{row["profile_path"]}'
            response = requests.get(img_path)
            img = Image.open(BytesIO(response.content))
            result = DeepFace.analyze(img_path, actions=['race'], enforce_detection=False)

            if isinstance(result, list):
                result_dict["profile_race"] = result[0]["dominant_race"]
            else:
                result_dict["profile_race"] = result["dominant_race"]
    except Exception as e:
        print(f'Error analyzing image for row {row["id"]}: {e}')

    # # Infer race from name
    # try:
    #     name_df = pd.DataFrame({'name': [row['name']]}, index=[0])
    #     inferred_race = pred_census_ln(name_df, 'name')
    #     result_dict["name_race"] = inferred_race['race'].values[0]
    # except Exception as e:
    #     print(f'Error analyzing name for row {row["id"]}: {e}')

    # append the results to the results DataFrame
    results_df = results_df.append(result_dict, ignore_index=True)

    return results_df

# Initialize or load the results DataFrame
write_file = 'person_tmdb_image_race.csv'
results_file_path = file_path + write_file

if os.path.exists(results_file_path):
    # Load the existing results file
    results_df = pd.read_csv(results_file_path)
else:
    # Create an empty DataFrame to store the results
    results_df = pd.DataFrame(columns=['id', 'name', 'profile_path', 'profile_race', 'name_race'])

# Process each row in the data frame using a standard loop
for _, row in data.iterrows():
    results_df = process_row(row, results_df)

    # increment the counter
    persons_analyzed += 1

    # print progress
    print(f"Persons Analyzed: {persons_analyzed}/{total_persons}")

    # write the results DataFrame to a CSV file every 100 iterations
    if persons_analyzed % 100 == 0:
        results_df.to_csv(results_file_path, index=False)

# write any remaining results to the file
if persons_analyzed % 100 != 0:
    results_df.to_csv(results_file_path, index=False)

# print newline character after completion
results_df.to_csv(results_file_path, index=False)
print()



Streaming output truncated to the last 5000 lines.
Persons Analyzed: 75202/96464
Persons Analyzed: 75203/96464
Persons Analyzed: 75204/96464
Persons Analyzed: 75205/96464
Persons Analyzed: 75206/96464
Persons Analyzed: 75207/96464
Persons Analyzed: 75208/96464
Persons Analyzed: 75209/96464
Persons Analyzed: 75210/96464
Persons Analyzed: 75211/96464
Persons Analyzed: 75212/96464
Persons Analyzed: 75213/96464
Persons Analyzed: 75214/96464
Persons Analyzed: 75215/96464
Persons Analyzed: 75216/96464
Persons Analyzed: 75217/96464
Persons Analyzed: 75218/96464
Persons Analyzed: 75219/96464
Persons Analyzed: 75220/96464
Persons Analyzed: 75221/96464
Persons Analyzed: 75222/96464
Persons Analyzed: 75223/96464
Persons Analyzed: 75224/96464
Persons Analyzed: 75225/96464
Persons Analyzed: 75226/96464
Persons Analyzed: 75227/96464
Persons Analyzed: 75228/96464
Persons Analyzed: 75229/96464
Persons Analyzed: 75230/96464
Persons Analyzed: 75231/96464
Persons Analyzed: 75232/96464
Persons Analyzed: 7

Action: race: 100%|██████████| 1/1 [00:00<00:00, 14.87it/s]


Persons Analyzed: 80202/96464
Persons Analyzed: 80203/96464


Action: race: 100%|██████████| 1/1 [00:00<00:00, 16.09it/s]


Persons Analyzed: 80204/96464


...

Persons Analyzed: 96445/96464
Persons Analyzed: 96446/96464
Persons Analyzed: 96447/96464
Persons Analyzed: 96448/96464
Persons Analyzed: 96449/96464
Persons Analyzed: 96450/96464
Persons Analyzed: 96451/96464
Persons Analyzed: 96452/96464
Persons Analyzed: 96453/96464
Persons Analyzed: 96454/96464
Persons Analyzed: 96455/96464


Action: race: 100%|██████████| 1/1 [00:00<00:00, 13.25it/s]


Persons Analyzed: 96456/96464
Persons Analyzed: 96457/96464
Persons Analyzed: 96458/96464
Persons Analyzed: 96459/96464
Persons Analyzed: 96460/96464
Persons Analyzed: 96461/96464
Persons Analyzed: 96462/96464


Action: race: 100%|██████████| 1/1 [00:00<00:00, 14.21it/s]


Persons Analyzed: 96463/96464
Persons Analyzed: 96464/96464



In [6]:
results_df.to_csv(results_file_path, index=False)


In [7]:
### Open Google Drive
# import pandas as pd
# from deepface import DeepFace
# from ethnicolr import pred_census_ln
# import requests
# from PIL import Image
# from io import BytesIO
# from google.colab import drive

# # Mount your Google Drive
# drive.mount('/content/drive')

# # Path to the CSV file on Google Drive
# file_path = '/content/drive/MyDrive/Projects/scrotten_movies/'
# read_file = 'person_data.csv'

# # Read the CSV file into a DataFrame
# data = pd.read_csv(file_path+read_file)

# # counter variables
# total_persons = len(data)
# persons_analyzed = 0

# def process_row(row):
#     result_dict = {
#         'id': row['id'],
#         'name': row['name'],
#         'profile_path': row['profile_path'],
#         'profile_race': None,
#         'name_race': None,
#     }

#     # Infer race from image
#     try:
#         if pd.notna(row['profile_path']):
#             img_path = f'https://image.tmdb.org/t/p/w300_and_h450_bestv2/{row["profile_path"]}'
#             response = requests.get(img_path)
#             img = Image.open(BytesIO(response.content))
#             result = DeepFace.analyze(img_path, actions=['race'], enforce_detection=False)

#             if isinstance(result, list):
#                 result_dict["profile_race"] = result[0]["dominant_race"]
#             else:
#                 result_dict["profile_race"] = result["dominant_race"]
#     except Exception as e:
#         print(f'Error analyzing image for row {row["id"]}: {e}')

#     # Infer race from name
#     try:
#         name_df = pd.DataFrame({'name': [row['name']]}, index=[0])
#         inferred_race = pred_census_ln(name_df, 'name')
#         result_dict["name_race"] = inferred_race['race'].values[0]
#     except Exception as e:
#         print(f'Error analyzing name for row {row["id"]}: {e}')

#     return result_dict

# # create an empty DataFrame to store the results
# results_df = pd.DataFrame(columns=['id', 'name', 'profile_path', 'profile_race', 'name_race'])

# # Process each row in the data frame using a standard loop
# for _, row in data.iterrows():
#     result = process_row(row)

#     # append the results to the results DataFrame
#     results_df = results_df.append(result, ignore_index=True)

#     # increment the counter
#     persons_analyzed += 1

#     # print progress
#     print(f"Persons Analyzed: {persons_analyzed}/{total_persons}")

#     # write the results DataFrame to a CSV file after every iteration
#     write_file = 'person_race.csv'
#     results_df.to_csv(file_path+write_file, index=False)

# # print newline character after completion
# print()


In [8]:
# ### Rewrotten to have profile and name be single pool
# import pandas as pd
# from deepface import DeepFace
# from ethnicolr import pred_census_ln
# import requests
# from PIL import Image
# from io import BytesIO

# # Path to the CSV file on Google Drive
# file_path = '/content/drive/MyDrive/Projects/scrotten_movies/'
# read_file = 'person_data.csv'

# # Read the CSV file into a DataFrame
# data = pd.read_csv(file_path+read_file)

# # counter variables
# total_persons = len(data)
# persons_analyzed = 0

# def process_row(row):
#     result_dict = {
#         'id': row['id'],
#         'name': row['name'],
#         'profile_path': row['profile_path'],
#         'profile_race': None,
#         'name_race': None,
#     }

#     # Infer race from image
#     try:
#         if pd.notna(row['profile_path']):
#             img_path = f'https://image.tmdb.org/t/p/w300_and_h450_bestv2/{row["profile_path"]}'
#             response = requests.get(img_path)
#             img = Image.open(BytesIO(response.content))
#             result = DeepFace.analyze(img_path, actions=['race'], enforce_detection=False)

#             if isinstance(result, list):
#                 result_dict["profile_race"] = result[0]["dominant_race"]
#             else:
#                 result_dict["profile_race"] = result["dominant_race"]
#     except Exception as e:
#         print(f'Error analyzing image for row {row["id"]}: {e}')

#     # Infer race from name
#     try:
#         name_df = pd.DataFrame({'name': [row['name']]}, index=[0])
#         inferred_race = pred_census_ln(name_df, 'name')
#         result_dict["name_race"] = inferred_race['race'].values[0]
#     except Exception as e:
#         print(f'Error analyzing name for row {row["id"]}: {e}')

#     return result_dict

# # create an empty DataFrame to store the results
# results_df = pd.DataFrame(columns=['id', 'name', 'profile_path', 'profile_race', 'name_race'])

# # Process each row in the data frame using a standard loop
# for _, row in data.iterrows():
#     result = process_row(row)

#     # append the results to the results DataFrame
#     results_df = results_df.append(result, ignore_index=True)

#     # increment the counter
#     persons_analyzed += 1

#     # print progress
#     print(f"Persons Analyzed: {persons_analyzed}/{total_persons}")

#     # write the results DataFrame to a CSV file after every iteration
#     write_file = 'person_race.csv'
#     # file_path = '/content/drive/MyDrive/Projects/scrotten_movies/'
#     # read_File = 'person_data.csv'
#     results_df.to_csv(file_path+write_file, index=False)
#     # print(results_df)

# # print newline character after completion
# print()


# Run Locally as Data Cleanup

In [2]:
import pandas as pd

# Load the data
df = pd.read_csv('./data/person_tmdb_image_race.csv')

# Keep only the desired columns
df = df[['id', 'name', 'profile_path', 'profile_race']]

# Remove rows where 'profile_path' is missing
df = df.dropna(subset=['profile_path'])

# Rename the desired columns
df = df.rename(columns={'profile_path': 'image_path', 'profile_race': 'image_race'})

# Write the DataFrame back to the CSV
df.to_csv('./data/person_tmdb_image_race.csv', index=False)
